In [1]:
import numpy as np
import pandas as pd
import os
import glob
import zipfile

In [5]:

root_dir = globals()['_dh'][0] #using this because code is ran from within jupyter nb
data_dir = f"{root_dir}\data"
extension = ".zip"

for item in os.listdir(data_dir): # loop through items in dir
    if item.endswith(extension): # check for ".zip" extension
        file_name = data_dir + "\\" + item # get full path of files
        zip_ref = zipfile.ZipFile(file_name) # create zipfile object
        zip_ref.extractall(data_dir) # extract file to dir
        zip_ref.close() # close file
        #os.remove(file_name) # delete zipped file

path = f"{os.getcwd()}\**\*.csv"

#prepare arrays for loading multiple csv->df and later concatenating
arr_najemi_posli = []
arr_najemi_delistavb = []
arr_prodaje_posli = []
arr_prodaje_delistavb = []
arr_prodaje_zemljisca = []

sifrant_skip = False
separator = ";"

for fname in glob.glob(path, recursive = True):
    
    if "sifranti" in fname and not sifrant_skip:
        sifrant = pd.read_csv(fname, encoding="utf-8", sep=separator, dtype=object)
        sifrant_skip = True
      
    elif "_KUP_" in fname:
        if "posli" in fname:
            df = pd.read_csv(fname, index_col=None, header=0, sep=separator, on_bad_lines='skip', dtype=object)
            arr_prodaje_posli.append(df)
        elif "delistavb" in fname:
            df = pd.read_csv(fname, index_col=None, header=0, sep=separator, on_bad_lines='skip', dtype=object)
            arr_prodaje_delistavb.append(df)
        elif "zemljisca" in fname:
            df = pd.read_csv(fname, index_col=None, header=0, sep=separator, on_bad_lines='skip', dtype=object)
            arr_prodaje_zemljisca.append(df)  
    
    #on_bad_lines skips lines that are faulty; contain more/less separators than col number
    elif "_NAJ_" in fname:
        if "posli" in fname:
            df = pd.read_csv(fname, index_col=None, header=0, sep=separator, on_bad_lines='skip', dtype=object)
            arr_najemi_posli.append(df)
        elif "delistavb" in fname:
            df = pd.read_csv(fname, index_col=None, header=0, sep=separator, on_bad_lines='skip', dtype=object)
            arr_najemi_delistavb.append(df)
    
    #--------------------------------------------
    #removes files after data has been loaded
    os.remove(fname)

walk = list(os.walk(data_dir))
for path, _, _ in walk[::-1]:
    if len(os.listdir(path)) == 0:
        os.rmdir(path)

#group year dfs into combined df per category
df_najemi_posli = pd.concat(arr_najemi_posli, axis=0, ignore_index=True)
df_najemi_delistavb = pd.concat(arr_najemi_delistavb, axis=0, ignore_index=True)
df_prodaje_posli = pd.concat(arr_prodaje_posli, axis=0, ignore_index=True)
df_prodaje_delistavb = pd.concat(arr_prodaje_delistavb, axis=0, ignore_index=True)
df_prodaje_zemljisca = pd.concat(arr_prodaje_zemljisca, axis=0, ignore_index=True)

df_dict = {
"sifrant": sifrant,
"najemi_posli": df_najemi_posli,
"najemi_delistavb": df_najemi_delistavb,
"prodaje_posli": df_prodaje_posli,
"prodaje_delistavb": df_prodaje_delistavb,
"prodaje_zemljisca": df_prodaje_zemljisca}

#saves combined files in output folder -> source for Power BI report
#step can be skipped if loading data to Dash app

#path = f"{os.getcwd()}\output\\"

#for key, value in df_dict.items():
#    value.to_csv(f"{path}{key}.csv")
